<a href="https://colab.research.google.com/github/inspire007/KaggleCompetitions/blob/main/Titanic_Machine_Learning_from_Disaster/Titanic_Machine_Learning_from_Disaster_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

data = pd.read_csv('train.csv')

#split X,y
X = data.iloc[:, [2,4,5,6,7,11]].values
y = data.iloc[:, 1].values

num_features = [0,2,3,4]
cat_features = [1,5]

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

models = {
    'LogisticRegression': LogisticRegression(),
    'RandomForest': RandomForestClassifier(n_estimators=500, random_state=42),
    'DecistionTree': DecisionTreeClassifier(),
    'SVC': SVC(kernel='rbf', random_state=42)
}

selected_model = False
max_acc = 0

for name,model in models.items():
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  cfm = confusion_matrix(y_test, y_pred)
  acc = accuracy_score(y_test, y_pred)
  if acc > max_acc:
    max_acc = acc
    selected_model = model
  print(f'{name} accuracy: {acc}')
  print(cfm)
  print('\n')

print('Selected model: ', selected_model)
print('Max accuracy: ', max_acc)

#train full dataset with selected model
X = preprocessor.fit_transform(X)
selected_model.fit(X,y)

#now predict given data sample
data = pd.read_csv('test.csv')
X_test_actual = data.iloc[:, [1,3,4,5,6,10]].values
X_test_actual = preprocessor.transform(X_test_actual)

y_pred_actual = selected_model.predict(X_test_actual)
pid = data['PassengerId'].values

np.savetxt('output.csv', np.column_stack((pid,y_pred_actual)), header="PassengerId,Survived", fmt ="%d", delimiter=',', comments='')

LogisticRegression accuracy: 0.8071748878923767
[[114  20]
 [ 23  66]]


RandomForest accuracy: 0.8026905829596412
[[115  19]
 [ 25  64]]


DecistionTree accuracy: 0.726457399103139
[[105  29]
 [ 32  57]]


SVC accuracy: 0.820627802690583
[[120  14]
 [ 26  63]]


Selected model:  SVC(random_state=42)
Max accuracy:  0.820627802690583
